In [ ]:
import datetime
import numpy as np
import pandas as pd
from pprint import pprint

import great_expectations as gx
from great_expectations.data_context import EphemeralDataContext

from matplotlib import pyplot as plt

%matplotlib inline

# Download and Explore Data

In [ ]:
# https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

In [ ]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet -O data/yellow_tripdata_2019-01.parquet

In [ ]:
# Read a data sample 
df = pd.read_parquet('../data/yellow_tripdata_2019-01.parquet').sample(10000)

# Prepare 'test.csv' for an example in Validate Data 
df.to_csv('../data/test.csv')

In [ ]:
df.head(3).T

In [ ]:
df.info()

In [ ]:
plt.hist(df['payment_type'])

In [ ]:
plt.hist(df['passenger_count'])

In [ ]:
plt.hist(df['trip_distance'])

In [ ]:
plt.hist(df['total_amount'])

# Create a Data Context 

## Ephemeral Data Contex

An Ephemeral Data Context is an in-memory Data Context that is not intended to persist beyond the current Python session. However, if you decide that you would like to save its contents for future use you can do so by converting it to a Filesystem Data Context ([docs](https://docs.greatexpectations.io/docs/oss/guides/setup/configuring_data_contexts/instantiating_data_contexts/instantiate_data_context))

In [ ]:
context = gx.get_context()

In [ ]:
context

In [ ]:
if isinstance(context, EphemeralDataContext):
    print("It's Ephemeral!")

In [ ]:
# Convert the Ephemeral Data Context into a Filesystem Data Context

context = context.convert_to_file_context()

## Filesystem Data Context 

In [ ]:
from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir="..")

# Same as
# context = gx.get_context(project_root_dir="..")

In [ ]:
context

# Connect to Data

## Use Validator (simple)

In [ ]:
validator = context.sources.pandas_default.read_parquet(
    '../data/yellow_tripdata_2019-01.parquet', 
)

In [ ]:
validator.head()

## Use Datasource 

In [ ]:
context.datasources

In [ ]:
# A Data Source provides a standard API for accessing and interacting with data from a wide variety of source systems.

# DEBUG: uncomment if you need to clear list for Datasources
context.datasources.clear()

datasource = context.sources.add_pandas_filesystem(
    name='local_dir', 
    base_directory="../data"
)
print("DATASOURCE:\n", datasource)

# The batching_regex should max file names in the data_directory
asset = datasource.add_parquet_asset(
    name="parquet_asset",
    batching_regex=r"yellow_tripdata_(?P<year>\d{4})-(?P<month>\d{2}).parquet",
    order_by=["year", "month"],
)
print("ASSET:\n", asset)

batch_request = asset.build_batch_request(options={"year": "2019", "month": "01"})
# options = asset.batch_request_options
# print("OPTIONS:\n", options)


In [ ]:
print(context.datasources["local_dir"])


In [ ]:
validator = context.get_validator(
    datasource_name="local_dir",
    data_asset_name="yellow_tripdata_2019-01.parquet",
    batch_request=batch_request
)

validator

In [ ]:
validator.head()

# Create Expectations

In [ ]:
validator.expect_column_values_to_not_be_null("tpep_pickup_datetime")

In [ ]:
validator.expect_column_values_to_be_between(
    "passenger_count", 
    min_value=1, 
    max_value=6
)

In [ ]:
validator.expect_column_values_to_be_between(
    column='payment_type',  
    min_value=0, 
    max_value=3, 
    mostly=0.9
)


In [ ]:
validator.expect_column_values_to_be_of_type(
    column='payment_type', 
    type_='int64'
)

# Save Expectation Suite

In [ ]:
validator.save_expectation_suite('../gx/expectations/first_expectation_suite.json')

# Validate Data

In [ ]:
# # DEBUG

# validation_context.sources.delete("taxi_test_datasource")
# del validation_context
# del test_data_validator


In [ ]:
validation_context = gx.get_context(context_root_dir="../gx")

In [ ]:
validation_context.list_expectation_suite_names()

In [ ]:
validation_suite = validation_context.get_expectation_suite("first_expectation_suite")
validation_suite

In [ ]:
test = pd.read_csv("../data/test.csv")

test_data_validator = (
    validation_context.sources.add_pandas("taxi_test_datasource")
    .read_dataframe(
       test, 
       asset_name="taxi_test", 
       batch_metadata={"type": "test"})
  )


In [ ]:
test_data_validator.expectation_suite


In [ ]:
test_data_validator.expectation_suite_name = "first_expectation_suite"

In [ ]:
test_data_validator.expectation_suite

In [ ]:
test_data_validator.expectation_suite

In [ ]:
checkpoint = validation_context.add_or_update_checkpoint(
    name="my_taxi_validator_checkpoint", 
    validator=test_data_validator,
    expectation_suite_name="first_expectation_suite"
)

checkpoint_result = checkpoint.run()

In [ ]:
checkpoint_result

In [ ]:
# validation_context.build_data_docs()

In [ ]:
validation_context.open_data_docs()